## NMF = Not Monday night Football !

In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
from random import randint
import numpy as np
from sklearn.decomposition import NMF
import pandas as pd
import random
from matplotlib import pyplot as plt
%matplotlib inline
import csv

# User Input

In [2]:
#importing ratings and movies csv files
PATH2 = "ratings.csv"
PATH3 = "movies.csv"
ratings, movies_ind = pd.read_csv(PATH2), pd.read_csv(PATH3)

In [3]:
# create an empty array the length of number of movies in system
user_ratings = np.zeros(9724)

In [4]:
#format ratings dataframe
del ratings['timestamp']
ratings.set_index(['userId','movieId'], inplace=True)
ratings = ratings.unstack(0)

In [5]:
ratings_count = ratings.count(axis=1) #count the number of ratings for each movie as a measure of popularity
top = pd.DataFrame(ratings_count.sort_values(ascending = False).head(10)) #create a dataframe of the top 20 most popular movies

In [6]:
top.reset_index(inplace=True)

In [7]:
# get a list of top movie titles from movie id
top_movies = []
for x in top['movieId']:
    top_movies.append(movies_ind[movies_ind['movieId']== x])

## Of the following movies, rate all that you have seen on a scale of 1-5. 
## If you have not seen a movie, rate 0.

In [8]:
#creates a list of ratings for the prompted movies
user_input = []
for i in range(0,10):
    answer = int(input("How would you rate " + str(top_movies[i]['title'])))
    user_input.append(answer)

How would you rate 314    Forrest Gump (1994)
Name: title, dtype: object2
How would you rate 277    Shawshank Redemption, The (1994)
Name: title, dtype: object3
How would you rate 257    Pulp Fiction (1994)
Name: title, dtype: object2
How would you rate 510    Silence of the Lambs, The (1991)
Name: title, dtype: object3
How would you rate 1939    Matrix, The (1999)
Name: title, dtype: object4
How would you rate 224    Star Wars: Episode IV - A New Hope (1977)
Name: title, dtype: object2
How would you rate 418    Jurassic Park (1993)
Name: title, dtype: object3
How would you rate 97    Braveheart (1995)
Name: title, dtype: object4
How would you rate 507    Terminator 2: Judgment Day (1991)
Name: title, dtype: object2
How would you rate 461    Schindler's List (1993)
Name: title, dtype: object3


In [9]:
# collect the indexes for each film in the user input array
top_movies_index = []
for x in range(0,10):
    top_movies_index.append(top_movies[x].index)

In [10]:
# inputs user rating into large array (9,000+ count) at appropriate indexes
for i in range(0,10):
    user_ratings[top_movies_index[i]] = user_ratings[i]

In [11]:
user_ratings

array([0., 0., 0., ..., 0., 0., 0.])

# NMF Modeling

In [12]:
PATH2 = "ratings.csv"
PATH3 = "movies.csv"
ratings, movies_ind = pd.read_csv(PATH2), pd.read_csv(PATH3)
del ratings['timestamp']
ratings.set_index(['movieId','userId'], inplace=True)
ratings = ratings.unstack(0)
ratings = ratings.fillna(0)
ratings = ratings["rating"]
#ratings = ratings.transpose()


In [13]:
ratings.head(3)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
R = pd.DataFrame(ratings)
# model assumes R ~ PQ'
model = NMF(n_components=5, init='random', random_state=10)
model.fit(R)

P = model.components_  # Movie feature
Q = model.transform(R)  # User features
#print(Q)

#print(model.reconstruction_err_)

nR = np.dot(Q,P) ### Reconstructed model
#print(nR)

###########Delete the crap below later 
#query = np.zeros(9724) ###This comes from Glynis array the size = # movies
#rr= 2
#for i in range(0,100):
   # print(i)
  #  query[i] = rr
  #  if i == 50:
      #  rr = 4
########################Delete this after we have user input 

query= user_ratings
query = query.reshape(1,-1)
t=model.transform(query)
model.transform(query)
outcome = np.dot(t,P)

##### Now we Select the best movies to offer the user
#ind = np.argpartition(outcome[0], -5)[-5:] 
#Select = ind[0:5][randint(0, 4)]

In [35]:
outcome=pd.DataFrame(outcome)

In [36]:
outcome = outcome.transpose()

In [37]:
outcome['movieId'] = movies_ind['movieId']

In [38]:
outcome = outcome.rename(columns={0:'rating'})

In [39]:
top =outcome.sort_values(by='rating',ascending=False).head(10)

top_movies = []
for x in top['movieId']:
   top_movies.append(movies_ind[movies_ind['movieId']== x])

top_movies

[   movieId             title                                       genres
 0        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy,
       movieId                      title                    genres
 6477    52885  Paprika (Papurika) (2006)  Animation|Mystery|Sci-Fi,
       movieId                   title genres
 6479    52952  This Is England (2006)  Drama,
       movieId                 title genres
 6480    52967  Away from Her (2006)  Drama,
       movieId              title                genres
 6481    52973  Knocked Up (2007)  Comedy|Drama|Romance,
       movieId             title                genres
 6482    52975  Hairspray (2007)  Comedy|Drama|Musical,
       movieId                  title                  genres
 6483    53000  28 Weeks Later (2007)  Horror|Sci-Fi|Thriller,
       movieId                                  title  \
 6484    53022  Wheels on Meals (Kuai can che) (1984)   
 
                            genres  
 6484  Action|Comedy|Crime|R

In [40]:
# select a random movie from top five options
# we should filter by genre next
Select = top_movies[0:5][randint(0, 4)]

In [41]:
Select

,movieId,title,genres
6481,52973,Knocked Up (2007),Comedy|Drama|Romance
